In [54]:
from bs4 import BeautifulSoup as BSoup
import re
import requests
import csv
import json
url = "http://quotes.toscrape.com/tag/inspirational/page/"
columns=['id','author','quote','tags','quote_length','born_date','born_location','author_url'] #Columns for CSV header

In [69]:
authorSet=dict()
dataSet=list()
page=1
nextPage=True
uid=0

In [70]:
while (nextPage):
    
    print(url+str(page))
    response = requests.get(url+str(page))
    source = BSoup(response.content)    
    print(source.find('title').get_text())
    
    if source.find('ul','pager').find('li','next'):
        txtNext = source.find('ul','pager').find('li','next').find('a').get_text()
        print(f"Processing {page} {nextPage} {txtNext}") 
    else:
        txtNext=None
    
    if txtNext and re.findall(r".*(Next).*",txtNext)[0]=="Next":    
        nextPage=True
    else:
        nextPage=False
        
    print(f"Processing {page} {nextPage}") 
    
    for quotes in source.find_all('div','quote'):
        quote = quotes.find(attrs={'itemprop':'text'}).get_text().strip()
        author = quotes.find(attrs={'class':'author'}).get_text().strip()
        tags = quotes.find(attrs={'itemprop':'keywords'}).get('content').strip()
        authorUrl = quotes.find(href=re.compile(r"/author/")).get('href')
        
        if authorUrl:
            print(authorUrl)
            authorKey = author.replace('.','_').replace(' ','_').strip()
            print(authorKey)
            
        if authorUrl and authorKey not in authorSet.keys():
            authorUrl = "http://quotes.toscrape.com"+authorUrl
            print(authorUrl)
            source_author = BSoup(requests.get(authorUrl).content)
            bornDate = source_author.find(attrs={'class':'author-born-date'}).get_text().strip()
            bornLocation = source_author.find(attrs={'class':'author-born-location'}).get_text().replace('in','').strip()
            authorSet[authorKey]={'name':author,'url':authorUrl,'date':bornDate,'location':bornLocation}
        else:
            print(f"Author ({authorKey}) details already found!")
            
        uid+=1    
        dataSet.append([uid,author,quote,tags.replace(',','|'),len(quote),
                        authorSet[authorKey]['date'],
                        authorSet[authorKey]['location'],
                        authorSet[authorKey]['url']            
        ]) 
    page+=1

http://quotes.toscrape.com/tag/inspirational/page/1
Quotes to Scrape
Processing 1 True Next →
Processing 1 True
/author/Albert-Einstein
Albert_Einstein
http://quotes.toscrape.com/author/Albert-Einstein
/author/Marilyn-Monroe
Marilyn_Monroe
http://quotes.toscrape.com/author/Marilyn-Monroe
/author/Thomas-A-Edison
Thomas_A__Edison
http://quotes.toscrape.com/author/Thomas-A-Edison
/author/Marilyn-Monroe
Marilyn_Monroe
Author (Marilyn_Monroe) details already found!
/author/Elie-Wiesel
Elie_Wiesel
http://quotes.toscrape.com/author/Elie-Wiesel
/author/J-K-Rowling
J_K__Rowling
http://quotes.toscrape.com/author/J-K-Rowling
/author/George-Eliot
George_Eliot
http://quotes.toscrape.com/author/George-Eliot
/author/C-S-Lewis
C_S__Lewis
http://quotes.toscrape.com/author/C-S-Lewis
/author/Martin-Luther-King-Jr
Martin_Luther_King_Jr_
http://quotes.toscrape.com/author/Martin-Luther-King-Jr
/author/Helen-Keller
Helen_Keller
http://quotes.toscrape.com/author/Helen-Keller
http://quotes.toscrape.com/tag/ins

In [71]:
authorSet

{'Albert_Einstein': {'name': 'Albert Einstein',
  'url': 'http://quotes.toscrape.com/author/Albert-Einstein',
  'date': 'March 14, 1879',
  'location': 'Ulm, Germany'},
 'Marilyn_Monroe': {'name': 'Marilyn Monroe',
  'url': 'http://quotes.toscrape.com/author/Marilyn-Monroe',
  'date': 'June 01, 1926',
  'location': 'The United States'},
 'Thomas_A__Edison': {'name': 'Thomas A. Edison',
  'url': 'http://quotes.toscrape.com/author/Thomas-A-Edison',
  'date': 'February 11, 1847',
  'location': 'Milan, Ohio, The United States'},
 'Elie_Wiesel': {'name': 'Elie Wiesel',
  'url': 'http://quotes.toscrape.com/author/Elie-Wiesel',
  'date': 'September 30, 1928',
  'location': 'Sighet, Romania'},
 'J_K__Rowling': {'name': 'J.K. Rowling',
  'url': 'http://quotes.toscrape.com/author/J-K-Rowling',
  'date': 'July 31, 1965',
  'location': 'Yate, South Gloucestershire, England, The United Kgdom'},
 'George_Eliot': {'name': 'George Eliot',
  'url': 'http://quotes.toscrape.com/author/George-Eliot',
  'd

In [72]:
dataSet

[[1,
  'Albert Einstein',
  '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'inspirational|life|live|miracle|miracles',
  131,
  'March 14, 1879',
  'Ulm, Germany',
  'http://quotes.toscrape.com/author/Albert-Einstein'],
 [2,
  'Marilyn Monroe',
  "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'be-yourself|inspirational',
  111,
  'June 01, 1926',
  'The United States',
  'http://quotes.toscrape.com/author/Marilyn-Monroe'],
 [3,
  'Thomas A. Edison',
  "“I have not failed. I've just found 10,000 ways that won't work.”",
  'edison|failure|inspirational|paraphrased',
  65,
  'February 11, 1847',
  'Milan, Ohio, The United States',
  'http://quotes.toscrape.com/author/Thomas-A-Edison'],
 [4,
  'Marilyn Monroe',
  "“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good 

## Write to CSV

In [73]:
def writeto_csv(data,filename,columns):
    with open(filename,'w+',newline='',encoding="UTF-8") as file:
        writer = csv.DictWriter(file, fieldnames=columns)
        writer.writeheader()
        writer = csv.writer(file)
        for element in data:
            writer.writerows([element])

### Create CSV file from dataSet

In [74]:
writeto_csv(dataSet,'quotes_inspirational.csv',columns)

### Write to JSON - author

In [75]:
with open("quotes_author_inspirational.json", "w") as file:
    json.dump(authorSet, file, indent=4, sort_keys=False)